# End-To-End Test V1

This jupyter notebook complements the existing `e2e_test_v1.py` to help people debug e2e test failures.

### Instruction
1. First, do `just up` from `models/ml_infra_core` in a terminal. The logs from `launch-gateway-v1-dev` and `launch-service-builder-dev` are available here
2. Make a copy of this notebook so we keep the template clean and unchanged
3. Just follow the notebook and execute cells that you think are relevant to what you want to test

### Tips
- To create new cells, select the border of a cell 
    - press "b" to create a cell below the selected cell
    - press "a" to create a cell above the selected cell
- To execute a cell
    - press "Shift+Enter" to execute the current and move on to the next cell
    - press "Ctrl+Enter" to execute the current and stay at the current cell (useful if you want to run the same cell multiple times)
- Run things on a devbox speeds up the dev/test iteration cycle 

### Troubleshooting guide
#### If you see 
```bash
ml_infra_core-launch-gateway-v1-dev-1       | kubernetes_asyncio.client.exceptions.ApiException: (401)
ml_infra_core-launch-gateway-v1-dev-1       | Reason: Unauthorized
```
- Do `just down` and `just up` again and the issue should be gone.

#### If you see 
```bash
ml_infra_core-launch-gateway-v1-dev-1       |   File "uvloop/loop.pyx", line 2016, in uvloop.loop.Loop.create_connection
ml_infra_core-launch-gateway-v1-dev-1       | OSError: [Errno 113] No route to host
```
Try creating new endpoints by executing `create_model_endpoint` and then run `delete_existing_endpoints(users=(user,))`


In [ ]:
from e2e_test_v1 import *

user = "juitse.hung-e2e-test"
# Options:
# - model-endpoint-simple-[a]sync
# - model-endpoint-custom-image-[a]sync
# - model-endpoint-runnable-image-[a]sync
endpoint_name = f"model-endpoint-simple-sync"

In [ ]:
# Wait for gateway to be ready before moving on to other cells
ensure_gateway_ready()

In [ ]:
# Optional: Delete existing endpoints for the current user
delete_existing_endpoints(users=(user,))

In [ ]:
# Feel free to create all of the model bundles. This cell takes about 6 seconds to run.
get_or_create_model_bundle(CREATE_MODEL_BUNDLE_REQUEST_SIMPLE, user, "v1")
get_or_create_model_bundle(CREATE_MODEL_BUNDLE_REQUEST_CUSTOM_IMAGE, user, "v1")
get_or_create_model_bundle(CREATE_MODEL_BUNDLE_REQUEST_RUNNABLE_IMAGE, user, "v2")

In [ ]:
"""
TODO: Change the request depending on which endpoint type you want to create
Options: 
- CREATE_[A]SYNC_MODEL_ENDPOINT_REQUEST_SIMPLE
- CREATE_[A]SYNC_MODEL_ENDPOINT_REQUEST_CUSTOM_IMAGE
- CREATE_[A]SYNC_MODEL_ENDPOINT_REQUEST_RUNNABLE_IMAGE
"""
create_model_endpoint(CREATE_SYNC_MODEL_ENDPOINT_REQUEST_SIMPLE, user)

In [ ]:
# Run this cell till your endpoint is ready
endpoints = list_model_endpoints(user)
ready_endpoints = [endpoint for endpoint in endpoints if endpoint["status"] == "READY"]
print(f"User {user} Current num endpoints: {len(endpoints)}, num ready endpoints: {len(ready_endpoints)}")
print(json.dumps(endpoints, indent=2))
assert len(ready_endpoints) >= 1 # TODO: Feel free to manually change how many endpoints you expect to be available 

In [ ]:
""" 
TODO: Change the update endpoint request depending on the type of endpoint you want to test
Options: 
- UPDATE_MODEL_ENDPOINT_REQUEST_SIMPLE
- UPDATE_MODEL_ENDPOINT_REQUEST_CUSTOM_IMAGE
- UPDATE_MODEL_ENDPOINT_REQUEST_RUNNABLE_IMAGE
Also remember to change the endpoint name to match the endpoint you want to update
Options: 
"""
update_model_endpoint(endpoint_name, UPDATE_MODEL_ENDPOINT_REQUEST_SIMPLE, user)

In [ ]:
# See check if the pods are running
!kubectl get pods -n scale-deploy -l user_id={user}

In [ ]:
# Run this command if you want to see logs from the pod
# !kubectl logs -n scale-deploy pods/<pod-name> -f

In [ ]:
# If you want to ssh into the pod, run this command on a separate terminal
# kubectl exec -n scale-deploy -it <pod-name> -- /bin/bash
# If you want to ssh into a specific container inside the pod, run this command on a separate terminal
# kubectl exec -it <pod-name> -c <container-name> -- /bin/bash

### For sync endpoint (and all endpoints after LIRA), you can try to forward a port from a pod to your local machine and send request to it
You need to **run this command in a separate terminal**

`kubectl port-forward -n scale-deploy pod/<pod-name> <local-port>:<remote-port>` 

E.g. `kubectl port-forward -n scale-deploy pod/<pod-name> 5000:5000` 

Note: This is a bit flaky and doesn't always work. But when I works it's really helpful!

In [ ]:
# This only works if you've run the port forwrading command in the previous cell
response = requests.post(
    f"http://localhost:5000/predict",
    json={
        "args": {"y": 1},
        "url": None,
        "return_pickled": False,
    },
    headers={"Content-Type": "application/json"},
    auth=requests.auth.HTTPBasicAuth("test-user-id", ""),
    timeout=DEFAULT_NETWORK_TIMEOUT_SEC,
)
print(response.json())